In [33]:
import pandas as pd
from functools import partial
import multiprocessing
from multiprocessing import Pool


In [7]:
from langdetect import detect

In [24]:
from langdetect import DetectorFactory

In [25]:
DetectorFactory.seed = 0

In [2]:
pd.set_option('display.max_columns', None)

In [17]:
# thanks to https://stackoverflow.com/questions/4182603/how-to-convert-a-string-to-utf-8-in-python
def make_unicode(input):
    if type(input) != unicode:
        input =  input.decode('utf-8')
        return input
    else:
        return input

In [3]:
chats_beta = pd.read_csv("../data/sample/chats_beta.csv")

In [4]:
chats_beta.head(50)

,chat_id,chat_title,emoji_list,sender_first_name,sender_id,sender_is_bot,sender_last_name,sender_username,text,timestamp
0,1122948260,WanchainCHAT - Wanchain.org Chat Group,[],Lorenzo,382511212,False,None,Takeshisan1992,Bitcoin going green,1521866182
1,1215859110,GDPNEWS,[],的过,470863699,False,路人,None,None,1524040209
2,1224810332,Bitxoxo,[],LevelNet ICO Help Support,559829592,False,None,None,None,1524237453
3,1224911847,EOSPrice,[],Abidin,530357558,False,Gul,None,23 i yhink,1525023914
4,1249433091,BKBT币快报官方中文社区,[],羊,585351960,False,舌浩阔,weioxun,None,1524934157
5,1146170349,Binance English,[],Digili,390589152,False,Digili,None,Alsrt,1524443188
6,1175254323,CryptoRunner - group,[],Harris,547212882,False,Harrison,okodo309,liking the look of ICON atm,1521850066
7,1119581721,TRON (TRX) - Main Fan Group,[],Crypto,485502609,False,O,None,When is it estimated to finish?,1520704721
8,1224911847,EOSPrice,[':grinningsquintingface:'],A,527026845,False,A,None,Thought means kisses xxx 😆,1524225829
9,1146170349,Binance English,[],ScreeN,510784727,False,007,ScreeN007,BTC on the way to Mars,1521105984


In [22]:
detect(make_unicode(chats_beta.iloc[10]['chat_title']))

u'en'

In [13]:
detect(u"今一はお前さん")

u'ja'

create a column with language name

In [27]:
def markDataFrameByChunks(df, num, col):
    ordNums = np.array(sorted(df[col].unique(),reverse=False))
    ordRange = range(0, len(ordNums))
    chunksDF = pd.DataFrame({'colName': ordNums, 'theirNum': ordRange}, index = ordRange)
    chunksDF['chunk'] = pd.cut(chunksDF['theirNum'], num)
    return chunksDF

In [65]:
# assign chunk numbers
chats_betaWithChunks = markDataFrameByChunks(chats_beta,3,'chat_title')
tmpDF = chats_beta.merge(chats_betaWithChunks, left_on = 'chat_title', right_on = 'colName', how = 'left')
tmpDF.drop(['colName','theirNum'], axis = 1, inplace = True)
print tmpDF.shape

(500, 11)


In [69]:
def parallel_lang_detect(colname, df):
    #print df.shape
    print df[1].columns
    #lang_dict = {}
    lang_list = []
    for index, row in df[1].iterrows():
        lang_dict = {}
        lang_dict['index_orig'] = index
        lang_dict['lang_orig'] = detect(make_unicode(row[colname]))
        lang_list.append(lang_dict)
    return lang_list

In [70]:
%%time
#dfByChunk = dfByChunk.groupby('chunk')
dfByChunk = tmpDF.groupby('chunk')
func = partial(parallel_lang_detect, "chat_title")
if __name__ == '__main__':
    p = Pool(processes=3)
    print 'defined workers'
    result = (p.map(func, dfByChunk))
    p.close()
    try:
        p.terminate()
    except WindowsError:
        pass
    p.join()
    dfByChunk = None
    print 'closed pool'
p.terminate()

Index([u'chat_id', u'chat_title', u'emoji_list', u'sender_first_name',
       u'sender_id', u'sender_is_bot', u'sender_last_name', u'sender_username',
       u'text', u'timestamp', u'chunk'],
      dtype='object')
Index([u'chat_id', u'chat_title', u'emoji_list', u'sender_first_name',
       u'sender_id', u'sender_is_bot', u'sender_last_name', u'sender_username',
       u'text', u'timestamp', u'chunk'],
      dtype='object')Index([u'chat_id', u'chat_title', u'emoji_list', u'sender_first_name',
       u'sender_id', u'sender_is_bot', u'sender_last_name', u'sender_username',
       u'text', u'timestamp', u'chunk'],
      dtype='object')

defined workers
closed pool
CPU times: user 27.3 ms, sys: 42.6 ms, total: 70 ms
Wall time: 3.56 s


In [74]:
tmpDF = None

In [72]:
%%time
# unpack list with outputs
unpackedDF = pd.DataFrame()
for res in result:
    #print res
    tmpDF = pd.DataFrame(res)
    unpackedDF = pd.concat([unpackedDF, tmpDF])

CPU times: user 9.77 ms, sys: 413 µs, total: 10.2 ms
Wall time: 9.93 ms


In [78]:
unpackedDF.set_index('index_orig', inplace = True);

In [79]:
unpackedDF.head()

,lang_orig
index_orig,
2,so
3,ro
4,zh-cn
5,en
6,fr


In [80]:
chats_beta_with_lang = chats_beta.merge(unpackedDF, left_index = True, right_index = True)

In [81]:
chats_beta_with_lang.head()

,chat_id,chat_title,emoji_list,sender_first_name,sender_id,sender_is_bot,sender_last_name,sender_username,text,timestamp,lang_orig
0,1122948260,WanchainCHAT - Wanchain.org Chat Group,[],Lorenzo,382511212,False,None,Takeshisan1992,Bitcoin going green,1521866182,en
1,1215859110,GDPNEWS,[],的过,470863699,False,路人,None,None,1524040209,de
2,1224810332,Bitxoxo,[],LevelNet ICO Help Support,559829592,False,None,None,None,1524237453,so
3,1224911847,EOSPrice,[],Abidin,530357558,False,Gul,None,23 i yhink,1525023914,ro
4,1249433091,BKBT币快报官方中文社区,[],羊,585351960,False,舌浩阔,weioxun,None,1524934157,zh-cn


In [82]:
chats_beta = None
unpackedDF = None